### Review contents download from Turnto UGC feed
Date: 10/17/2019 <br>
Author: Shawn Jung <br>
Description: This notebook download the review contents JSON file from Turnto's Feed server <br>
For detailed specifcation of the UGC Feed, visit https://docs.turnto.com/en/data-feeds/ugc-feed.html <br>
Avi Aryan's code was super helpful - https://www.codementor.io/aviaryan/downloading-files-from-urls-in-python-77q3bs0un

Load related libraries, define variables and prepare donwload URLs to use

In [1]:
import requests
import yaml
import json 
import pandas as pd

review_file_name = 'turnto_ugc.json'
data_path = './data/'
feed_source = "qa" # Assuming we have 'qa' and 'live' environment distinction, and marked in the config yml 

In [ ]:
with open('turnto_ugc_config_sample.yml', 'r') as yml_file:
    config_doc = yaml.load(yml_file, Loader = yaml.SafeLoader)

site_key = config_doc[feed_source]["sitekey"]
auth_key = config_doc[feed_source]["authkey"]

review_contents_url = 'https://www.turnto.com/static/export/{}/{}/turnto-ugc.js'.format(site_key, auth_key)

Define functions to check downloadability and download over web

In [ ]:
def is_downloadable(url):
    h = requests.head(url, allow_redirects=True)
    header = h.headers
    content_type = header.get('content-type')
    if 'text/xml' in content_type.lower():
        return True
    if 'text' in content_type.lower():
        return False
    if 'html' in content_type.lower():
        return False
    else:
        return True

def file_download(url, file_name):
    if is_downloadable(url):
        r = requests.get(url, allow_redirects=True)
        open(data_path+file_name, 'wb').write(r.content)
        print(file_name + " is downloaded")
    else: 
        print("The URL is not downloadable")

Now we download the feed and load into memory. We also make sure we didn't get the in-progress file

In [ ]:
file_download(review_contents_url, review_file_name)
ugc_raw =  json.load(open('./data/turnto_ugc.json', encoding='utf8'))

In [1]:
feed_status = review_raw['exportStart'] # it should be 'complete' 
print(feed_status)

The UGC file includes various user-generated contents. Let's assume we just need 'comment' type texts for analysis, so process them in a csv file

In [ ]:
comment_list = ugc_raw['comments']

In [ ]:
i = 0 
review_list_len = len(review_list)
''' iterating to flatten sku from nested JSON item info structure '''
while i < review_list_len:
    review_list[i]['sku_no'] = review_list[i]['item']['sku']
    review_list[i]['product_title'] = review_list[i]['item']['title']
    i += 1 

In [ ]:
comment_list = pd.DataFrame(comment_list, columns=['id', 'sku_no', 'product_title', 'title','text','rating','locale','dateTime'])
comment_list.head()
comment_list.to_csv('./data/comment_list.csv')